In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import KBinsDiscretizer
import math
import warnings

In [2]:
cna= pd.read_csv ('after_mrmr/after_mrmr_CNA.csv')# will import datset

label= pd.read_csv ('after_mrmr/after_mrmr_label_all.csv')# will import datset

clinical= pd.read_csv ('after_mrmr/after_mrmr_clinical_all.csv')

gene= pd.read_csv ('after_mrmr/after_mrmr_geneexp.csv')

In [3]:
rna = pd.read_csv('data_mRNA_median_Zscores.csv')

In [4]:
cna=cna.rename(columns={'Unnamed: 0': 'Patient ID'})
cna=cna.rename(columns={'Patient ID': 'Patient_ID'})

In [5]:
label=label.rename(columns={'Patient ID': 'Patient_ID'})

In [6]:
clinical=clinical.drop(columns=['Unnamed: 0'])

In [7]:
clinical=clinical.rename(columns={'PATIENT_ID': 'Patient_ID'})

In [8]:
gene_join=gene.iloc[0:,0:1]

In [15]:
gene_join=gene_join.rename(columns={'Unnamed: 0': 'Patient_ID'})

In [16]:
cna_1904=gene_join.set_index('Patient_ID').join(cna.set_index('Patient_ID'))
merg_cna= pd.merge(left=cna, right=gene_join, left_on='Patient_ID', right_on='Patient_ID')
cna_1904=cna_1904.fillna(0.001)
merg_cna

,Patient_ID,DDX11,KMO,LINC00229,DBF4,SSSCA1,DBR1,CYP7A1,CYBA,PGLS,...,LINC00358,SPDYE8P,SUCLG2,PGA5,LINC00515,LINC00659,LINC00299,SULT4A1,LINC00305,PCSK4
0,MB-0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MB-0039,1.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,MB-0045,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
3,MB-0046,-1.0,1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
4,MB-0048,0.0,0.0,-1.0,0.0,1.0,0.0,1.0,-1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,MB-5332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1900,MB-5417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1901,MB-5614,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
1902,MB-6329,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
label_1904=gene_join.set_index('Patient_ID').join(label.set_index('Patient_ID'))
merged_label = pd.merge(left=gene_join, right=label, left_on='Patient_ID', right_on='Patient_ID')
label_1904.fillna(0.001)
merged_label

,Patient_ID,OS_MONTHS
0,MB-0362,0.0
1,MB-0346,0.0
2,MB-0386,0.0
3,MB-0574,0.0
4,MB-0503,0.0
...,...,...
1899,MB-5465,0.0
1900,MB-5453,0.0
1901,MB-5471,1.0
1902,MB-5127,1.0


In [24]:
clinical_1904=gene_join.set_index('Patient_ID').join(clinical.set_index('Patient_ID'))
merged_clinical = pd.merge(left=gene_join, right=clinical, left_on='Patient_ID', right_on='Patient_ID')
clinical_1904=clinical_1904.fillna(0.001)
merged_clinical = merged_clinical.fillna(0.01)
clinical_1904

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,AGE_AT_DIAGNOSIS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY
Patient_ID,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,52.79,0.0,2.0,0.00,0.0,1.0,1.0,0.0,0.0
MB-0346,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,32.61,0.0,5.0,3.00,0.0,1.0,0.0,0.0,0.0
MB-0386,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,66.83,1.0,2.0,1.00,1.0,0.0,1.0,0.0,1.0
MB-0574,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,71.21,1.0,2.0,0.00,1.0,0.0,1.0,0.0,1.0
MB-0503,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,60.77,1.0,2.0,1.00,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,69.83,0.0,4.0,2.00,0.0,0.0,1.0,0.0,1.0
MB-5453,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,36.99,0.0,0.0,2.00,0.0,1.0,1.0,0.0,1.0
MB-5471,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,67.54,1.0,2.0,0.00,1.0,0.0,0.0,0.0,0.0


In [20]:
print(clinical.shape)
print(gene.shape)
print(cna.shape)
print(label.shape)


(2509, 20)
(1904, 401)
(2173, 201)
(2506, 2)


In [26]:
clinical_1904=clinical_1904.fillna(0.01)
clinical_1904=clinical_1904.replace(to_replace = 'UNDEF', value = 0.01)  
clinical_1904

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,AGE_AT_DIAGNOSIS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY
Patient_ID,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,52.79,0.0,2.0,0.00,0.0,1.0,1.0,0.0,0.0
MB-0346,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,32.61,0.0,5.0,3.00,0.0,1.0,0.0,0.0,0.0
MB-0386,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,66.83,1.0,2.0,1.00,1.0,0.0,1.0,0.0,1.0
MB-0574,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,71.21,1.0,2.0,0.00,1.0,0.0,1.0,0.0,1.0
MB-0503,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,60.77,1.0,2.0,1.00,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,69.83,0.0,4.0,2.00,0.0,0.0,1.0,0.0,1.0
MB-5453,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,36.99,0.0,0.0,2.00,0.0,1.0,1.0,0.0,1.0
MB-5471,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,67.54,1.0,2.0,0.00,1.0,0.0,0.0,0.0,0.0


In [ ]:
maxjoin=clinical.set_index('Patient_ID').join(gene.set_index('Patient_ID'))

merged_inner = pd.merge(left=clinical, right=gene, left_on='Patient_ID', right_on='Patient_ID')


In [ ]:
maxjoin.to_csv(r'maxjoin1.csv', index = True)

In [10]:
merged_inner.to_csv(r'merged_inner.csv', index = True)

NameError: name 'merged_inner' is not defined

In [11]:
merged_inner

NameError: name 'merged_inner' is not defined

In [27]:
print(clinical_1904.shape)
print(gene.shape)
print(cna_1904.shape)
print(label_1904.shape)

(1904, 19)
(1904, 401)
(1904, 200)
(1904, 1)


In [28]:
print(np.where(pd.isnull(clinical_1904)))
print(np.where(pd.isnull(gene)))
print(np.where(pd.isnull(cna_1904)))
print(np.where(pd.isnull(label_1904)))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [29]:
print(np.where(clinical_1904.applymap(lambda x: x == ' ')))
print(np.where(gene.applymap(lambda x: x == ' ')))
print(np.where(cna_1904.applymap(lambda x: x == ' ')))
print(np.where(label_1904.applymap(lambda x: x == ' ')))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [30]:
print(np.where(pd.isna(clinical_1904)))
print(np.where(pd.isna(gene)))
print(np.where(pd.isna(cna_1904)))
print(np.where(pd.isna(label_1904)))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [138]:
rnaa=rna
rnaa=rnaa.drop(columns=['Entrez_Gene_Id'])
rnaa=rnaa.T
rnaaa=rnaaa.fillna(0)

In [32]:
merg1=clinical_1904.merge(cna_1904, left_on='Patient_ID', right_on='Patient_ID')
merg1

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,...,LINC00358,SPDYE8P,SUCLG2,PGA5,LINC00515,LINC00659,LINC00299,SULT4A1,LINC00305,PCSK4
Patient_ID,,,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,...,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0
MB-0346,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,...,-1.0,1.0,-1.0,1.0,2.0,1.0,1.0,0.0,-1.0,1.0
MB-0386,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
MB-0574,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,...,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,0.0
MB-0503,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
MB-5453,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
MB-5471,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0


In [35]:
gene=gene.set_index('Unnamed: 0')

In [36]:
gene

,VPS13D,CF887450,T65755,USP3,ANK1,B3GALT4,TRAF1,BM983985,NR2C2AP,PDCD7,...,MPP5,EIF4E,LOC340073,AKAP2,AP2A1,AX747098,FKBP7,SLC7A6OS,ING4,HHATL
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,...,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
MB-0346,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
MB-0386,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
MB-0574,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
MB-0503,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0
MB-5453,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
MB-5471,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,...,2.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0


In [37]:
clinical_1904.to_csv(r'clinical_1904.csv', index = False)
cna_1904.to_csv(r'cna_1904.csv', index = False)
label_1904.to_csv(r'label_1904.csv', index = False)
gene.to_csv(r'gene_1904.csv', index = False)




In [38]:
clinical_1904

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,AGE_AT_DIAGNOSIS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY
Patient_ID,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,52.79,0.0,2.0,0.00,0.0,1.0,1.0,0.0,0.0
MB-0346,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,32.61,0.0,5.0,3.00,0.0,1.0,0.0,0.0,0.0
MB-0386,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,66.83,1.0,2.0,1.00,1.0,0.0,1.0,0.0,1.0
MB-0574,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,71.21,1.0,2.0,0.00,1.0,0.0,1.0,0.0,1.0
MB-0503,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,60.77,1.0,2.0,1.00,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,69.83,0.0,4.0,2.00,0.0,0.0,1.0,0.0,1.0
MB-5453,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,36.99,0.0,0.0,2.00,0.0,1.0,1.0,0.0,1.0
MB-5471,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,67.54,1.0,2.0,0.00,1.0,0.0,0.0,0.0,0.0


In [163]:
print(np.where(pd.isnull(merg1)))
print(np.where(pd.isna(merg1)))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [50]:
gene=gene.reset_index()
gene=gene.rename(columns={'Unnamed: 0': 'Patient_ID'})

In [51]:
gene

,Patient_ID,VPS13D,CF887450,T65755,USP3,ANK1,B3GALT4,TRAF1,BM983985,NR2C2AP,...,MPP5,EIF4E,LOC340073,AKAP2,AP2A1,AX747098,FKBP7,SLC7A6OS,ING4,HHATL
0,MB-0362,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,...,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
1,MB-0346,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2,MB-0386,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
3,MB-0574,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
4,MB-0503,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,MB-5465,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0
1900,MB-5453,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1901,MB-5471,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,...,2.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0
1902,MB-5127,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [52]:
Early_fusion2=merg1.merge(gene, left_on='Patient_ID', right_on='Patient_ID')
Early_fusion2=Early_fusion2.set_index('Patient_ID')
Early_fusion2

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,...,MPP5,EIF4E,LOC340073,AKAP2,AP2A1,AX747098,FKBP7,SLC7A6OS,ING4,HHATL
Patient_ID,,,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,...,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
MB-0346,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
MB-0386,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,...,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
MB-0574,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
MB-0503,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,...,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0
MB-5453,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
MB-5471,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,...,2.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0


In [53]:
print(np.where(pd.isnull(Early_fusion2)))
print(np.where(pd.isna(Early_fusion2)))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [54]:
Early_fusion2.to_csv(r'for_fusion.csv', index = False)

In [201]:
rnaareset=rnaa.reset_index()

In [203]:
rnaareset

,index,0,1,2,3,4,5,6,7,8,...,24358,24359,24360,24361,24362,24363,24364,24365,24366,24367
0,Hugo_Symbol,RERE,RNF165,CD049690,BC033982,PHF7,CIDEA,PAPD4,AI082173,SLC17A3,...,BX115874,BX107598,UGCGL1,VPS72,CSMD3,CC2D1A,CB986545,IGSF9,DA110839,FAM71A
1,MB-0362,-0.7232,-0.4613,0.2128,-2.1103,-0.3352,-0.0178,-0.7702,-0.4132,0.7594,...,-0.8111,1.1739,0.1423,-0.3022,-0.5244,-0.0023,-3.3223,0.4021,-0.9987,-1.1301
2,MB-0346,1.2351,0.3564,0.2147,0.468,-1.0641,-1.0424,0.0478,0.5041,-0.517,...,1.5124,-0.1363,0.3671,0.3318,-0.4342,-0.7585,0.1217,1.3014,0.4363,0.0333
3,MB-0386,-0.0081,-0.6809,0.5602,-0.2548,0.2557,3.288,-0.65,-0.3937,0.6686,...,-1.1462,-1.0747,0.62,0.4692,6.8829,0.0427,1.1943,0.7965,-1.1174,0.6638
4,MB-0574,-0.4468,-1.0575,0.3441,0.2517,-0.3009,-0.2676,2.1756,0.6297,-1.1013,...,-1.346,0.8866,0.2513,0.238,1.0408,0.2403,-0.765,-0.1714,-2.1531,-0.3571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900,MB-5465,-1.8168,3.5789,-0.0125,-2.5155,-0.9299,-0.7176,-1.0148,-0.7137,0.0198,...,-0.3053,0.0049,0.0557,-0.4101,0.2226,-0.2881,0.2719,0.3788,-0.2235,-0.2414
1901,MB-5453,1.1415,1.3437,0.7974,-1.614,-0.0615,-0.4365,0.6217,-1.4017,-0.3137,...,0.2095,0.3267,-1.1005,-1.5622,0.0711,2.5161,-0.5976,0.3216,1.1124,0.5541
1902,MB-5471,0.0233,0.5728,-0.6225,-0.8468,0.5209,-0.0384,0.5743,-2.6813,-0.6884,...,-0.0747,-0.5834,0.2296,-0.6457,0.119,-0.8334,0.9157,0.8192,0.1083,-0.9352
1903,MB-5127,-0.359,0.1729,1.0303,-1.0421,-0.034,-0.4044,2.4335,0.7242,0.674,...,-0.3553,-0.2826,-0.5897,-0.8265,-0.3201,-0.1287,-2.3114,-0.5763,1.7433,0.181


In [204]:
merged_final = pd.merge(left=Early_fusion2, right=rnaareset, left_on='Patient_ID', right_on='index')



In [206]:
merged_final.to_csv(r'merged_final.csv', index = False)

In [207]:
merged_final

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,...,24358,24359,24360,24361,24362,24363,24364,24365,24366,24367
0,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,...,-0.8111,1.1739,0.1423,-0.3022,-0.5244,-0.0023,-3.3223,0.4021,-0.9987,-1.1301
1,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,...,1.5124,-0.1363,0.3671,0.3318,-0.4342,-0.7585,0.1217,1.3014,0.4363,0.0333
2,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,...,-1.1462,-1.0747,0.62,0.4692,6.8829,0.0427,1.1943,0.7965,-1.1174,0.6638
3,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,...,-1.346,0.8866,0.2513,0.238,1.0408,0.2403,-0.765,-0.1714,-2.1531,-0.3571
4,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,...,-0.3579,-0.0699,0.1674,-0.1773,0.3272,-0.4779,-0.181,-0.2633,-0.5411,-1.2264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,...,-0.3053,0.0049,0.0557,-0.4101,0.2226,-0.2881,0.2719,0.3788,-0.2235,-0.2414
1900,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,...,0.2095,0.3267,-1.1005,-1.5622,0.0711,2.5161,-0.5976,0.3216,1.1124,0.5541
1901,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,...,-0.0747,-0.5834,0.2296,-0.6457,0.119,-0.8334,0.9157,0.8192,0.1083,-0.9352
1902,0.0,4.060,1.0,0.0,3.0,1.0,0,0.0,1.0,1.0,...,-0.3553,-0.2826,-0.5897,-0.8265,-0.3201,-0.1287,-2.3114,-0.5763,1.7433,0.181


In [208]:
print(np.where(pd.isnull(merged_final)))
print(np.where(pd.isna(merged_final)))

(array([ 971, 1084, 1271, 1319, 1407, 1577, 1609, 1695, 1799, 1887]), array([16442,  1864,  1940,  1943, 24987,  1891,  1864,  1940,  1754,
       12546]))
(array([ 971, 1084, 1271, 1319, 1407, 1577, 1609, 1695, 1799, 1887]), array([16442,  1864,  1940,  1943, 24987,  1891,  1864,  1940,  1754,
       12546]))


nan
nan
nan
